In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing

In [3]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"

films = pd.read_csv(film_csv, index_col=None, header=0)
display(films.head())

trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
display(trailers_meta.head())

,id,title,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties
0,60432,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д..."
1,64219,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес..."
2,65407,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл..."
3,75149,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""..."
4,75629,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...","{ ""Аудитория"" : [""для детей 13-16 лет"",""9 лет""..."


,Unnamed: 0,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,4,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,5,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,6,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,7,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [4]:
trailers_meta['properties'].to_list()
cols = ["Место действия", "Качества", "Время действия", "На основе", "Аудитория", "Настроение", "Поджанр", "Про что", "Тематика"]

places = []
qualities = []
times = []
mains = []
audiences = []
moods = []
subgenres = []
abouts = []
themes = []

for i in range(0, len(trailers_meta)):
    trailers_info = json.loads(trailers_meta.iloc[i]['properties'])
    
    try:
        place = trailers_info['Место действия']
    except:
        place = ['не определено']
    places.append(place)
    
    try:
        quality = trailers_info['Качества']
    except:
        quality = ['не определено']
    qualities.append(quality)
    
    try:
        time = trailers_info['Время действия']
    except:
        time = ['не определено']
    times.append(time)
    
    try:
        main = trailers_info['На основе']
    except:
        main = ['не определено']
    mains.append(main)
    
    try:
        audience = trailers_info['Аудитория']
    except:
        audience = ['не определено']
    audiences.append(audience)
    
    try:
        mood = trailers_info['Настроение']
    except:
        mood = ['не определено']
    moods.append(mood)
    
    try:
        subgenre = trailers_info['Поджанр']
    except:
        subgenre = ['не определено']
    subgenres.append(subgenre)

    try:
        about = trailers_info['Про что']
    except:
        about = ['не определено']
    abouts.append(about)
    
    try:
        theme = trailers_info['Тематика']
    except:
        theme = ['не определено']
    themes.append(theme)
    

df = pd.DataFrame({
    'content_id': trailers_meta['content_id'],
    'Place of act': places,
    'Qualities': qualities,
    'Time of act': times,
    'Based on': mains,
    'Audience': audiences,
    'Mood': moods,
    'Subgenre': subgenres,
    'About': abouts,
    'Theme': themes
})

display(df.head())
display(trailers_meta.head())

trailers_df = pd.merge(trailers_meta, df, left_on='content_id', right_on='content_id', how='left')
trailers_df = trailers_df.drop_duplicates(subset=['content_id'])
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

trailers_df.to_csv(trailers_csv)

,content_id,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,60432,[США],[дебютный],"[21-й век, новейшее время]",[оригинального сценария],"[для молодёжи, для взрослых]","[грустный, депрессивный, трогательный]","[Драмы, Комедийные]","[преследования, самозванцы, расследование, раз...","[развлекательные, психологический]"
1,64219,[не определено],"[зрелищные, жанровый, неожиданный финал]",[не определено],"[эпоса и легенд, оригинального сценария]","[для взрослых, для мужчин, для подготовленного...","[трагичные, мрачные, шокирующие, захватывающие...","[ужасы, триллер]","[преследования, страхи, дети, молодежь, монстр...",[развлекательные]
2,65407,[не определено],"[жанровый, дебютный]",[новейшее время],[оригинального сценария],"[для взрослых, для подготовленного зрителя, дл...","[захватывающие, жестокие, напряженные, мрачные...","[Драмы, Боевики, триллер]","[психотерапевты, кровь, тайна смерти, психолог...","[психологический, развлекательные]"
3,75149,"[большой город, США]","[жанровый, остросюжетные]",[20-й век],[комиксов],"[для мужчин, для женщин, для взрослых]","[захватывающие, авантюрный, жестокие, напряжен...","[Боевики, Драмы, криминальные]","[враги или вражда, женщины, антигерои, жертвы,...","[авантюрные, гангстерская]"
4,75629,"[вымышленная страна (город), большой город]","[частично анимационный, зрелищные, кассовые, о...",[21-й век],"[оригинального сценария, сериалов]","[для детей 13-16 лет, 9 лет, 8 лет, 7-12, 12 л...","[авантюрный, захватывающие, динамичный, весёлы...","[детские, Приключенческие, Фантастические, Дет...","[притяжение противоположностей, риск, преследо...","[развлекательные, образовательный]"


,Unnamed: 0,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,4,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,5,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,6,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,7,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,...,Theme_x,Place of act_y,Qualities_y,Time of act_y,Based on_y,Audience_y,Mood_y,Subgenre_y,About_y,Theme_y
0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},...,"['развлекательные', 'психологический']",[США],[дебютный],"[21-й век, новейшее время]",[оригинального сценария],"[для молодёжи, для взрослых]","[грустный, депрессивный, трогательный]","[Драмы, Комедийные]","[преследования, самозванцы, расследование, раз...","[развлекательные, психологический]"
1,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},...,['развлекательные'],[не определено],"[зрелищные, жанровый, неожиданный финал]",[не определено],"[эпоса и легенд, оригинального сценария]","[для взрослых, для мужчин, для подготовленного...","[трагичные, мрачные, шокирующие, захватывающие...","[ужасы, триллер]","[преследования, страхи, дети, молодежь, монстр...",[развлекательные]
2,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},...,"['психологический', 'развлекательные']",[не определено],"[жанровый, дебютный]",[новейшее время],[оригинального сценария],"[для взрослых, для подготовленного зрителя, дл...","[захватывающие, жестокие, напряженные, мрачные...","[Драмы, Боевики, триллер]","[психотерапевты, кровь, тайна смерти, психолог...","[психологический, развлекательные]"
3,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},...,"['авантюрные', 'гангстерская']","[большой город, США]","[жанровый, остросюжетные]",[20-й век],[комиксов],"[для мужчин, для женщин, для взрослых]","[захватывающие, авантюрный, жестокие, напряжен...","[Боевики, Драмы, криминальные]","[враги или вражда, женщины, антигерои, жертвы,...","[авантюрные, гангстерская]"
4,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}",...,"['развлекательные', 'образовательный']","[вымышленная страна (город), большой город]","[частично анимационный, зрелищные, кассовые, о...",[21-й век],"[оригинального сценария, сериалов]","[для детей 13-16 лет, 9 лет, 8 лет, 7-12, 12 л...","[авантюрный, захватывающие, динамичный, весёлы...","[детские, Приключенческие, Фантастические, Дет...","[притяжение противоположностей, риск, преследо...","[развлекательные, образовательный]"


In [9]:
# добавить зависимость от визуальных данных, на первое время сгодится и это

bad_ids = [165474, 211551, 219612, 220568, 354250, 414508, 415752, 424144, 424148, 424149, 424151, 424171, 437578, 437675, 443045, 443395, 443436, 446816, 447184, 447188, 447259, 447460, 447466, 447501, 447502, 447503, 447547, 447548, 447580, 447633, 447817, 447818, 447821, 447965, 447967, 447976, 448060, 448061, 448070, 448074, 448176, 448177, 448178, 448179, 448180, 448181, 448427, 448428, 448536, 448544, 448551, 448901, 448902, 449030, 449694, 449695, 450175, 450176, 450259, 450279, 450281, 450482, 450584, 450585, 450670, 450671, 450672, 450673, 450674, 450832, 450834, 450996, 450997, 451166, 451230, 451385, 451528, 451542, 451773, 452136, 452170, 452176, 452178]
def make_session(df, start_ind = 0, session_len = 10):
    used_inds = [start_ind]
    content_ids = [df.iloc[start_ind]['content_id']]
    time_of_act = df.iloc[start_ind]['Time of act_x'] #array
    emotions = json.loads(df.iloc[start_ind]['emotions'].replace("\'", "\""))
    emotions_neu = emotions['neutral'] #obj
    emotions_neg = emotions['negative'] #obj
    emotions_pos = emotions['positive'] #obj
    emotions_skip = emotions['skip'] #obj
    emotions_speech = emotions['speech'] #obj
    genre = df.iloc[start_ind]['genres'] #array
    mood = df.iloc[start_ind]['Mood_x'] #array
    about = df.iloc[start_ind]['About_x'] #array
    
    i = 0
    while i < len(df):
        if len(used_inds) == session_len:
            break
        if i not in used_inds:
            i_emo = json.loads(df.iloc[i]['emotions'].replace("\'", "\""))
            i_genre = df.iloc[i]['genres']
            i_mood = df.iloc[i]['Mood_x']
            i_about = df.iloc[i]['About_x']
            i_time = df.iloc[i]['Time of act_x']
            if ((emotions_neu * 0.6 < i_emo['neutral'] < emotions_neu * 1.4) and
                (emotions_neg * 0.6 < i_emo['negative'] < emotions_neg * 1.4) and
                (emotions_pos * 0.6 < i_emo['positive'] < emotions_pos * 1.4) and
                (emotions_skip * 0.6 < i_emo['skip'] < emotions_skip * 1.4) and
                (emotions_speech * 0.6 < i_emo['speech'] < emotions_speech * 1.4) and
                len(list(set(i_genre) & set(genre))) > 0 and
                len(list(set(i_mood) & set(mood))) > 0 and
                len(list(set(i_about) & set(about))) > 0 and
                len(list(set(i_time) & set(time_of_act))) > 0 and 
                df.iloc[i]['content_id'] not in bad_ids
               ):
                used_inds.append(i)
                content_ids.append(df.iloc[i]['content_id'])
                genre = list(set(i_genre) | set(genre))
                mood = list(set(i_mood) | set(mood))
                about = list(set(i_about) | set(about))
                time_of_act = list(set(i_time) | set(time_of_act))
                emotions_neu = np.mean([emotions_neu, i_emo['neutral']])
                emotions_neg = np.mean([emotions_neg, i_emo['negative']])
                emotions_pos = np.mean([emotions_pos, i_emo['positive']])
                emotions_skip = np.mean([emotions_skip, i_emo['skip']])
                emotions_speech = np.mean([emotions_speech, i_emo['speech']])
                i = 0
            else:
                i += 1
        else:
            i += 1
        
    return content_ids
    

sessions = []
j = 0
while j < len(trailers_df):
    print(j)
    session = make_session(trailers_df, j)
    if len(session) == 10:
        print("append", len(sessions))
        sessions.append(session)
    j += 1
    if len(sessions) > 50:
        break;
    
sess_df = pd.DataFrame(sessions)
display(sess_df.head())
sess_df.to_csv("/Volumes/Seagate/natasha-diploma/sessions.csv")

0
append 0
1
2
append 1
3
append 2
4
append 3
5
append 4
6
append 5
7
append 6
8
9
append 7
10
11
append 8
12
append 9
13
append 10
14
append 11
15
append 12
16
append 13
17
append 14
18
append 15
19
20
append 16
21
append 17
22
append 18
23
append 19
24
append 20
25
append 21
26
27
append 22
28
append 23
29
30
append 24
31
32
33
append 25
34
append 26
35
append 27
36
append 28
37
append 29
38
append 30
39
append 31
40
41
42
append 32
43
append 33
44
append 34
45
46
append 35
47
48
49
append 36
50
append 37
51
append 38
52
append 39
53
54
append 40
55
append 41
56
append 42
57
58
59
60
61
append 43
62
append 44
63
append 45
64
append 46
65
append 47
66
append 48
67
append 49
68
append 50


,0,1,2,3,4,5,6,7,8,9
0,60432,174457,184174,111847,172257,227812,250885,174558,212559,214967
1,65407,225043,138191,177902,176620,225371,190349,147590,181198,138245
2,75149,60432,133587,148584,186787,222725,212087,174369,423155,431690
3,75629,205372,186058,171392,160160,184174,172257,75630,226747,212559
4,75630,172257,219611,211554,248722,250977,216926,190967,415428,419254


In [12]:
trailers_meta.to_csv(trailers_csv)